<a href="https://colab.research.google.com/github/paolochikalo/datascience_work/blob/master/M5_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('./drive/My Drive/utilities/'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gc
import time
import math
import datetime
from math import log, floor

from utility_functions import reduce_mem_usage, read_csv_files_in_folder

from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from fbprophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import joblib
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
import sklearn

import warnings
warnings.filterwarnings("ignore")


!pip install ipython-autotime
%load_ext autotime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=3d8beda1a832eee1a0426f1cf8e7c16627eb1ccc6603d152bc009cd7e19bf228
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [ ]:
calendar, sales_train, sample_submission, sell_prices = read_csv_files_in_folder("./drive/My Drive/DATA/M5-Forecasting")

Reading files...
There are total 5 files in folder
calendar.csv
Mem. usage decreased to  0.12 Mb (41.9% reduction)
calendar has 1969 rows and 14 columns
sales_train_validation.csv
Mem. usage decreased to 95.00 Mb (78.7% reduction)
sales_train_validation has 30490 rows and 1919 columns
sample_submission.csv
Mem. usage decreased to  2.09 Mb (84.5% reduction)
sample_submission has 60980 rows and 29 columns
sell_prices.csv
Mem. usage decreased to 130.48 Mb (37.5% reduction)
sell_prices has 6841121 rows and 4 columns
time: 2min 34s


In [ ]:
print(sales_train.head())
calendar.head()
sell_prices.head()

                              id        item_id  ... d_1912 d_1913
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  ...      1      1
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  ...      0      0
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  ...      1      1
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  ...      7      2
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  ...      2      4

[5 rows x 1919 columns]


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


time: 135 ms


In [ ]:
sales_train_melt = pd.melt(sales_train, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='day', value_name='demand')

time: 18.2 s


In [ ]:
print(sales_train_melt.shape)
print(sales_train.shape)
sales_train_melt.head()

(58327370, 8)
(30490, 1919)


,id,item_id,dept_id,cat_id,store_id,state_id,day,demand
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


time: 52.3 ms


In [ ]:
sales_CA_1 = sales_train_melt[sales_train_melt.store_id == "CA_1"]
new_CA_1 = pd.merge(sales_CA_1, calendar, left_on="day", right_on="d", how="left")
new_CA_1 = pd.merge(new_CA_1, sell_prices, left_on=["store_id", "item_id", "wm_yr_wk"],right_on=["store_id", "item_id", "wm_yr_wk"], how="left")
new_CA_1["day_int"] = new_CA_1.day.apply(lambda x: int(x.split("_")[-1]))
new_CA_1["store_id"].unique()

array(['CA_1'], dtype=object)

time: 16.4 s


In [ ]:
print(new_CA_1.shape)
new_CA_1.head()

(5832737, 24)


,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,day_int
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,NaN,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,NaN,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,NaN,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,NaN,1
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,NaN,1


time: 63.2 ms


In [ ]:
new_CA_1.shape

(5832737, 24)

time: 9.02 ms


In [ ]:
# FILL MISSING price values with 0
print("Count NaN demand:", new_CA_1['demand'].isna().sum())
print(new_CA_1['sell_price'].isna().sum())
new_CA_1['sell_price'].fillna(0)
print(new_CA_1['sell_price'].isna().sum())

In [ ]:
#counting sales for CA_1 store
new_CA_1['sales'] = new_CA_1['sell_price']*new_CA_1['demand']
print(new_CA_1['sales'].head())
day_sales = new_CA_1.groupby(["day_int", "store_id"])[["sales", "demand"]].agg("sum")
day_sales.head()

In [ ]:
day_sum = new_CA_1.groupby("day_int")[["sell_price", "demand"]].agg("sum")

time: 2.69 s


In [ ]:
print(day_sum.shape)
day_sum.head()

(1913, 2)


,sell_price,demand
day_int,,
1,4912.0,4337
2,4912.0,4155
3,4912.0,2816
4,4912.0,3051
5,4912.0,2630


time: 28.3 ms


In [ ]:
day_sum=day_sum.reset_index()
print(day_sum.shape)
day_sum.head()

(1913, 3)


,day_int,sell_price,demand
0,1,4912.0,4337
1,2,4912.0,4155
2,3,4912.0,2816
3,4,4912.0,3051
4,5,4912.0,2630


time: 21.7 ms
